In [1]:
import warnings
from dotenv import load_dotenv
warnings.filterwarnings('ignore')
load_dotenv(override=True)

True

In [2]:
from typing import Optional, Dict, Any, List
from pydantic import BaseModel
from langchain_groq import ChatGroq
from selenium.webdriver.chrome.webdriver import WebDriver
from langgraph.types import Command
from langgraph.graph import StateGraph, START, END
import json

llm= ChatGroq(model="openai/gpt-oss-20b", temperature= 0)

class BrowserAgentState(BaseModel):
    user_query: Optional[str] = None      
    intent: Optional[List[Dict[str, Any]]] = None  # LLM output is a list of steps
    action_plan: Optional[List[Dict[str, Any]]] = None
    next_step: Optional[Dict[str, Any]] = None    # currently executing step
    current_step: int = 0                          # pointer in action_plan
    current_url: Optional[str] = None
    dom_snapshot: Optional[Dict[str, Any]] = None
    execution_trace: List[Dict[str, Any]] = []  
    error: Optional[str] = None
    

In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
# import pyautogui
from webdriver_manager.chrome import ChromeDriverManager


options = Options()
options.add_experimental_option("detach", True)
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()),
            options=options)
# driver.get("https://amazon.in")

In [4]:
# from selenium import webdriver
# from selenium.webdriver.chrome.service import Service
# from selenium.webdriver.chrome.options import Options
# from webdriver_manager.chrome import ChromeDriverManager

# chrome_options = Options()
# chrome_options.add_argument("--start-maximized")

# driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)


In [5]:
from langchain_core.messages import HumanMessage

def intent_parser_node(state: BrowserAgentState) -> BrowserAgentState:
    query = state.user_query
    prompt = f"""
You are an intent extraction assistant for a browser automation agent.
The user will give a natural language command about browsing the web.

Convert the command into a sequence of steps in JSON array format.
Each step must follow this structure:
{{
  "intent": "<one of: navigate, search, click, extract, login, custom>",
  "target": "<website or element if mentioned>",
  "query": "<search query or action text if applicable>"
}}

Rules:
- Always return a JSON array of steps.
- If the user mentions a website, use the canonical URL.
- For example:
  - "search on Wikipedia" → {{"intent": "navigate", "target": "Wikipedia", "query": "https://en.wikipedia.org"}}
  - "search on Google" → {{"intent": "navigate", "target": "Google", "query": "https://www.google.com"}}
- After navigating, add search, click, or extract steps as needed.

Here is the user command:
{query}
"""

    try:
        response = llm.invoke([HumanMessage(content=prompt)])
        raw_output= response.content  # raw JSON string here
        print("LLM raw output:", raw_output)  # 👈 debug
        state.intent = json.loads(raw_output)
    except Exception as e:
        state.error = str(e)
        return Command(goto="error_node")
    return Command(goto="router_node")

In [6]:
# # Create a test state
# state = BrowserAgentState(
#     user_query="search about India in wikipedia"
# )

# # Run only the intent parser node
# result_state = intent_parser_node(state)

# # Check output
# # print("LLM raw output:", result_state.intent)

In [7]:
def router_node(state: BrowserAgentState) -> Command:
    print("router_node")
    if state.action_plan is None:
        if not state.intent:
            state.error = "No intent found from parser"
            return Command(goto="error_node")
        state.action_plan = state.intent
        state.current_step = 0

    if state.current_step >= len(state.action_plan):
        return Command(goto=END)

    step = state.action_plan[state.current_step]
    state.next_step = step
    state.current_step += 1

    intent_type = step["intent"].lower()
    if intent_type == "navigate":
        return Command(goto="navigate_node")
    elif intent_type == "search":
        return Command(goto="search_node")
    elif intent_type == "click":
        return Command(goto="click_node")
    elif intent_type == "extract":
        return Command(goto="extract_node")
    elif intent_type == "login":
        return Command(goto="login_node")
    else:
        state.error = f"Unknown intent: {intent_type}"
        return Command(goto="error_node")


In [8]:
from langgraph.types import Command
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.action_chains import ActionChains
import time

def navigate_node(state: BrowserAgentState) -> Command:
    print("navigate_node")
    step = state.next_step
    url = step.get("query")

    try:
        # Use Selenium to navigate
        driver.get(url)
        state.current_url = url
        state.execution_trace.append({"action": "navigate", "target": step["target"], "query": url})
    except Exception as e:
        state.error = str(e)
        return Command(goto="error_node")

    # After execution, go back to router for next step
    return Command(goto="router_node")



In [9]:
# # Create a dummy state
# state = BrowserAgentState(
#     next_step={
#         "intent": "navigate",
#         "target": "Wikipedia",
#         "query": "https://en.wikipedia.org"
#     }
# )

# # Call the node
# result = navigate_node(state)

# # Check the state after execution
# print("Current URL:", state.current_url)
# print("Execution Trace:", state.execution_trace)
# print("Command returned:", result)

In [10]:
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()),
            options=options)
def search_node(state: BrowserAgentState) -> Command:
    step = state.next_step
    query = step.get("query")
    target = step.get("target")
    print("Query:", query)
    print("Target:", target)
    
    try:
        if target.lower() == "google":
            driver.get("https://www.google.com")
            selector = "input[name='q']"

            # Handle consent popup if present
            try:
                consent_btn = WebDriverWait(driver, 3).until(
                    EC.element_to_be_clickable((By.XPATH, "//button[contains(text(),'I agree') or contains(text(),'Accept all')]"))
                )
                consent_btn.click()
                time.sleep(1)
            except:
                pass  # No popup

            # Wait until search box is visible and enabled
            search_box = WebDriverWait(driver, 5).until(
                EC.presence_of_element_located((By.NAME, "q"))
            )
            
            search_box.clear()
            search_box.send_keys(query)
            search_box.send_keys(Keys.ENTER)  # simulate Enter key

            # Wait for results page to load
            WebDriverWait(driver, 5).until(lambda d: d.current_url != "https://www.google.com")

            state.execution_trace.append({
                "action": "search",
                "target": selector,
                "query": query,
                "url": driver.current_url
            })

        else:
            state.error = f"Unsupported target: {target}"
            return Command(goto="error_node")

    except Exception as e:
        state.error = f"Search failed: {e}"
        return Command(goto="error_node")

    return Command(goto="router_node")


In [11]:
# state = BrowserAgentState(
#     next_step={"query": "Selenium automation", "target": "google"}
# )
# command = search_node(state)
# if state.error:
#     print("Error:", state.error)
# else:
#     print("Execution Trace:", state.execution_trace)
#     print("Next Node:", command.goto)
# driver.close()

In [12]:
# from selenium.common.exceptions import (
#     NoSuchElementException,
#     TimeoutException,
#     ElementClickInterceptedException
# )
# from selenium.webdriver.support.ui import WebDriverWait
# from selenium.webdriver.support import expected_conditions as EC
# driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()),
#             options=options)
# def click_node(state: BrowserAgentState) -> Command:
#     print("click_node")
#     step = state.next_step
#     target = step.get("target")  # CSS selector of the element to click

#     try:
#         element = WebDriverWait(driver, 30).until(
#             EC.element_to_be_clickable((By.CSS_SELECTOR, target))
#         )
#         driver.execute_script("arguments[0].scrollIntoView(true);", element)
#         element.click()
#         state.execution_trace.append({"action": "click", "target": target})
#     except TimeoutException:
#         state.error = f"Timeout: Element with CSS selector '{target}' not clickable."
#         return Command(goto="error_node")
        
#     except NoSuchElementException:
#         state.error = f"Element with CSS selector '{target}' not found."
#         return Command(goto="error_node")
#     except ElementClickInterceptedException:
#         state.error = f"Element '{target}' was intercepted (popup or overlay)."
#         return Command(goto="error_node")
#     except Exception as e:
#         state.error = f"Click failed: {str(e)}"
#         return Command(goto="error_node")

#     return Command(goto="router_node")


In [13]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import (
    NoSuchElementException,
    TimeoutException,
    ElementClickInterceptedException
)
import time

# --- Dummy classes for testing ---
class BrowserAgentState:
    def __init__(self):
        self.next_step = {}
        self.execution_trace = []
        self.error = None

class Command:
    def __init__(self, goto=None):
        self.goto = goto

# --- Setup WebDriver ---
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")

driver = webdriver.Chrome(
    service=Service(ChromeDriverManager().install()),
    options=options
)

# --- Your click_node function ---

def click_node(state: BrowserAgentState) -> Command:
    print("click_node")
    step = state.next_step
    target_text = step.get("target")  # Here, the visible text of the link/button

    try:
        # Wait until the element with the link text is present
        element = WebDriverWait(driver, 30).until(
            EC.presence_of_element_located((By.LINK_TEXT, target_text))
        )

        driver.execute_script("arguments[0].scrollIntoView(true);", element)

        try:
            element.click()
        except Exception:
            # Fallback: JS click if normal click fails
            driver.execute_script("arguments[0].click();", element)

        state.execution_trace.append({"action": "click", "target": target_text})

    except TimeoutException:
        state.error = f"Timeout: Element with link text '{target_text}' not found in time."
        return Command(goto="error_node")

    except NoSuchElementException:
        state.error = f"Element with link text '{target_text}' not found."
        return Command(goto="error_node")

    except ElementClickInterceptedException:
        state.error = f"Element '{target_text}' was intercepted (popup or overlay)."
        return Command(goto="error_node")

    except Exception as e:
        state.error = f"Click failed: {str(e)}"
        return Command(goto="error_node")

    return Command(goto="router_node")





In [14]:
# driver.get("https://www.selenium.dev/documentation/webdriver/elements/interactions/")
# state = BrowserAgentState()
# state.next_step = {"target": "send keys"}  # visible text of the link
# result = click_node(state)

# print("Execution Trace:", state.execution_trace)
# print("Error:", state.error)
# print("Next Node:", result.goto)


In [15]:



# # The extract_node function you provided (or the improved version)
# def extract_node(state: BrowserAgentState, driver) -> Command:
#     print("extract_node")
#     step = state.next_step
#     target = step.get("target")

#     try:
#         element = driver.find_element(By.CSS_SELECTOR, target)
#         value = element.text
#         state.execution_trace.append({"action": "extract", "target": target, "value": value})
#     except NoSuchElementException:
#         state.error = f"Element with CSS selector '{target}' not found."
#         return Command(goto="error_node")
#     except Exception as e:
#         state.error = str(e)
#         return Command(goto="error_node")

#     return Command(goto="router_node")


In [16]:
import os
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from webdriver_manager.chrome import ChromeDriverManager

# --- Mock the required classes and state ---
class BrowserAgentState:
    def __init__(self, next_step, url):
        self.next_step = next_step
        self.url = url
        self.execution_trace = []
        self.error = None

class Command:
    def __init__(self, goto):
        self.goto = goto

# The robust extract_node function
def extract_node(state: BrowserAgentState, driver: webdriver) -> Command:
    print(f"Executing extract_node for URL: {state.url}")
    target = state.next_step.get("target")

    try:
        # Use explicit wait for the element to be visible
        wait = WebDriverWait(driver, 10)
        element = wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, target)))
        value = element.text
        state.execution_trace.append({"action": "extract", "target": target, "value": value})
        print(f"Successfully extracted: {value}")
    except (TimeoutException, NoSuchElementException):
        state.error = f"Element with CSS selector '{target}' not found or not visible within timeout."
        print(f"Error during extraction: {state.error}")
        return Command(goto="error_node")
    except Exception as e:
        state.error = str(e)
        print(f"General error during extraction: {state.error}")
        return Command(goto="error_node")

    return Command(goto="router_node")

# --- Test function for DemoQA with new selector ---
def run_simple_website_test():
    try:
        service = Service(ChromeDriverManager().install())
        driver = webdriver.Chrome(service=service)
        driver.maximize_window()
    except Exception as e:
        print(f"Failed to start WebDriver: {e}")
        return

    demo_qa_url = "https://demoqa.com/elements"
    
    # Updated target selector to be more specific
    # You may need to inspect the live site to find the most up-to-date selector
    target_selector = "div.main-header"
    
    state = BrowserAgentState(next_step={"target": target_selector}, url=demo_qa_url)

    # Navigate to the page
    driver.get(state.url)

    # Adding a longer wait for initial page load, or waiting for a more generic element
    try:
        # Wait for the main page content to be visible before looking for the specific element
        WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.CSS_SELECTOR, "div.body-content")))
    except TimeoutException:
        print("Timeout waiting for main body content. Proceeding anyway.")
    
    time.sleep(1) # Small pause for stability

    # Execute your function
    result = extract_node(state, driver)

    print("\n--- Test Results ---")
    print(f"Final Command: {result.goto}")
    if state.error:
        print(f"Error Message: {state.error}")
    else:
        print(f"Execution Trace: {state.execution_trace}")

    # Verify the result
    expected_text = "Elements"
    if not state.error and state.execution_trace and state.execution_trace[-1].get('value') == expected_text:
        print("Test Passed: Extracted text matches expected text.")
    else:
        print("Test Failed: Extracted text does not match.")

    # Clean up
    driver.quit()


run_simple_website_test()


WebDriverException: Message: target frame detached
  (failed to check if window was closed: disconnected: Unable to receive message from renderer)
  (Session info: chrome=140.0.7339.208)
Stacktrace:
	GetHandleVerifier [0x0xebc333+65459]
	GetHandleVerifier [0x0xebc374+65524]
	(No symbol) [0x0xcdd7c0]
	(No symbol) [0x0xccde79]
	(No symbol) [0x0xcccfaa]
	(No symbol) [0x0xceb4af]
	(No symbol) [0x0xcea7db]
	(No symbol) [0x0xd6b8e8]
	(No symbol) [0x0xd49bf6]
	(No symbol) [0x0xd1b38e]
	(No symbol) [0x0xd1c274]
	GetHandleVerifier [0x0x113eda3+2697763]
	GetHandleVerifier [0x0x1139ec7+2677575]
	GetHandleVerifier [0x0xee4194+228884]
	GetHandleVerifier [0x0xed49f8+165496]
	GetHandleVerifier [0x0xedb18d+192013]
	GetHandleVerifier [0x0xec47d8+99416]
	GetHandleVerifier [0x0xec4972+99826]
	GetHandleVerifier [0x0xeaebea+10346]
	BaseThreadInitThunk [0x0x76f6fcc9+25]
	RtlGetAppContainerNamedObjectPath [0x0x77d282ae+286]
	RtlGetAppContainerNamedObjectPath [0x0x77d2827e+238]


In [ ]:

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.get("https://www.google.com")
print(driver.title)
driver.quit()

Google


In [ ]:
def error_node(state: BrowserAgentState) -> Command:
    # Log the error
    error_info = state.error if state.error else "Unknown error occurred"
    print(f"[ERROR NODE] Execution stopped. Error: {error_info}")
    
    # Optionally, add to execution trace
    state.execution_trace.append({"action": "error", "message": error_info})
    
    # Stop the workflow
    return Command(goto="END")

In [ ]:
graph = StateGraph(BrowserAgentState)
graph.add_node("intent_parser_node",intent_parser_node)
graph.add_node("router_node",router_node)
graph.add_node("navigate_node",navigate_node)
graph.add_node("error_node", error_node)
graph.add_node("click_node",click_node)
graph.add_node("extract_node", extract_node)
graph.add_node("search_node", search_node)

graph.add_edge(START, "intent_parser_node")
graph.add_edge("intent_parser_node", "router_node")

browser_graph = graph.compile()



NameError: name 'extract_node' is not defined

Test part


In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
import time

# Set up the WebDriver
service = Service()
driver = webdriver.Chrome(service=service)

try:
    # Navigate to the website
    driver.get("https://www.demoblaze.com/")
    
    # Wait for dynamic content to load. Adjust time as necessary.
    time.sleep(5) 

    # Get the complete, rendered page source
    full_html = driver.page_source
    
    print("Fetched the complete HTML source.")

finally:
    driver.quit()

Fetched the complete HTML source.


In [ ]:
from bs4 import BeautifulSoup

# Use BeautifulSoup to parse the HTML
soup = BeautifulSoup(full_html, 'html.parser')

for tag in soup.find_all(['button', 'a']):
    tag.insert_before(f"[{tag.name.upper()}]")
    tag.insert_after(f"[/{tag.name.upper()}]")

clean_text = soup.get_text(separator='\n', strip=True)

In [ ]:
clean_text

"STORE\nNew message\n[BUTTON]\n×\n[/BUTTON]\nContact Email:\nContact Name:\nMessage:\n[BUTTON]\nClose\n[/BUTTON]\n[BUTTON]\nSend message\n[/BUTTON]\nSign up\n[BUTTON]\n×\n[/BUTTON]\nUsername:\nPassword:\n[BUTTON]\nClose\n[/BUTTON]\n[BUTTON]\nSign up\n[/BUTTON]\nLog in\n[BUTTON]\n×\n[/BUTTON]\nUsername:\nPassword:\n[BUTTON]\nClose\n[/BUTTON]\n[BUTTON]\nLog in\n[/BUTTON]\nAbout us\n[BUTTON]\n×\n[/BUTTON]\nVideo Player is loading.\n[BUTTON]\nPlay Video\n[/BUTTON]\n[BUTTON]\nPlay\n[/BUTTON]\n[BUTTON]\nMute\n[/BUTTON]\nCurrent Time\n0:00\n/\nDuration\n-:-\nLoaded\n:\n0%\nStream Type\nLIVE\n[BUTTON]\nSeek to live, currently playing live\nLIVE\n[/BUTTON]\nRemaining Time\n-\n0:00\n[BUTTON]\nPlayback Rate\n[/BUTTON]\n1x\n[BUTTON]\nChapters\n[/BUTTON]\nChapters\n[BUTTON]\nDescriptions\n[/BUTTON]\ndescriptions off\n, selected\n[BUTTON]\nCaptions\n[/BUTTON]\ncaptions settings\n, opens captions settings dialog\ncaptions off\n, selected\n[BUTTON]\nAudio Track\n[/BUTTON]\n[BUTTON]\nPicture-in-Picture

In [ ]:
def get_llm_response(user_query, clean_text):
    """
    Sends a prompt and a web page's text to an LLM and returns the response.
    (This is a simplified example)
    """
    llm_prompt = f"""
You are a web page analysis assistant.
Below is the extracted text from a website.

Website Content:
{clean_text}

User Query:
"{user_query}"

Task:
Identify the element (button, link, or text) that best matches the user's query.
Return your answer in JSON format with keys:
- "element_text" : the visible text of the element
- "reasoning" : a short explanation
"""
    response = llm.invoke(llm_prompt)
    return response.content if hasattr(response, "content") else str(response)
    

# Construct the prompt for the LLM
user_query = "Help me identify the price of nexus 6"
# llm_prompt = f"Given the following content from a website, answer the user's query.\n\nWebsite Content:\n{clean_text}\n\nUser Query: {user_query}"

# Use the LLM to choose the element
llm_recommendation = get_llm_response(user_query, clean_text)
print(llm_recommendation)

```json
{
  "element_text": "$650",
  "reasoning": "The price $650 is listed immediately after the Nexus 6 product link, indicating the cost of that item."
}
```


In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

# Start driver
driver = webdriver.Chrome()

# Step 1: Navigate to the site
driver.get("https://www.demoblaze.com/index.html")

# Step 2: Click on "Samsung galaxy s6"
product = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.LINK_TEXT, "Samsung galaxy s6"))
)
product.click()

# Step 3: Wait for modal to load
add_to_cart_button = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, "//a[text()='Add to cart']"))
)

# Step 4: Extract button HTML + text
button_text = add_to_cart_button.text
button_html = add_to_cart_button.get_attribute("outerHTML")

print("Button Text:", button_text)
print("Button HTML:", button_html)

# Optional: Click the button to test
# add_to_cart_button.click()

time.sleep(3)
driver.quit()


Button Text: Add to cart
Button HTML: <a href="#" onclick="addToCart(1)" class="btn btn-success btn-lg">Add to cart</a>


Testing


In [19]:
from pydantic import BaseModel
from typing import List, Dict, Any, Optional

class BrowserAgentState(BaseModel):
    user_query: Optional[str] = None
    intent: List[Dict[str, Any]] = []
    action_plan: List[Dict[str, Any]] = []
    next_step: Optional[Dict[str, Any]] = None
    current_step: int = 0
    current_url: Optional[str] = None
    dom_snapshot: Dict[str, Any] = {}
    execution_trace: List[Dict[str, Any]] = []
    error: Optional[str] = None

In [23]:
def intent_parser_node(state: BrowserAgentState) -> Command:
    query = state.user_query
    prompt = f"""
You are an intent extraction assistant for a browser automation agent.
The user will give a natural language command about browsing the web.

Convert the command into a sequence of steps in JSON array format.
Each step must follow this structure:
{{
  "intent": "<one of: navigate, search, click, extract, login, custom>",
  "target": "<website or element if mentioned>",
  "query": "<search query or action text if applicable>"
}}

Rules:
- Always return a JSON array of steps.
- If the user mentions a website, use the canonical URL.
- For example:
  - "search on Wikipedia" → {{"intent": "navigate", "target": "Wikipedia", "query": "https://en.wikipedia.org"}}
  - "search on Google" → {{"intent": "navigate", "target": "Google", "query": "https://www.google.com"}}
- After navigating, add search, click, or extract steps as needed.

Here is the user command:
{query}
"""

    try:
        response = llm.invoke([HumanMessage(content=prompt)])
        raw_output = response.content
        print("LLM raw output:", raw_output)

        # parse and **update state in-place**
        state.intent = json.loads(raw_output)
        print(f"[INTENT PARSER] Successfully parsed intent: {state.intent}")

    except Exception as e:
        state.error = str(e)
        return Command( goto="error_node")

    # MUST return Command, not state
    return Command( goto="router_node")


def router_node(state: BrowserAgentState) -> Command:
    print("=== router_node ===")
    print("state.intent:", state.intent)
    print("Current state:", state.__dict__)  # debug

    if not state.action_plan:
        if not state.intent:
            state.error = "No intent found from parser"
            print("[ROUTER ERROR] No intent found")
            return Command(goto="error_node")
        state.action_plan = state.intent.copy()
        state.current_step = 0
        print("[ROUTER] Initialized action_plan:", state.action_plan)

    if state.current_step >= len(state.action_plan):
        print("[ROUTER] All steps completed")
        return Command(goto=END)

    step = state.action_plan[state.current_step]
    print(f"[ROUTER] Executing step {state.current_step}: {step}")
    state.next_step = step

    intent_type = step.get("intent", "").strip().lower()
    allowed_intents = {"navigate", "search", "click", "extract", "login", "custom"}

    if intent_type not in allowed_intents:
        state.error = f"Unknown intent: {intent_type}"
        print("[ROUTER ERROR]", state.error)
        return Command(goto="error_node")

    # Don't increment here; increment after successful execution
    print(f"[ROUTER] Routing intent: {intent_type}")
    return Command(goto=f"{intent_type}_node")


In [24]:
state = BrowserAgentState(user_query="Go to Wikipedia and search for Python programming")



cmd_after_parser = intent_parser_node(state)
first_cmd = router_node(state)
print("First command:", first_cmd)

LLM raw output: [
  {
    "intent": "navigate",
    "target": "Wikipedia",
    "query": "https://en.wikipedia.org"
  },
  {
    "intent": "search",
    "target": "Wikipedia",
    "query": "Python programming"
  }
]
[INTENT PARSER] Successfully parsed intent: [{'intent': 'navigate', 'target': 'Wikipedia', 'query': 'https://en.wikipedia.org'}, {'intent': 'search', 'target': 'Wikipedia', 'query': 'Python programming'}]
=== router_node ===
state.intent: [{'intent': 'navigate', 'target': 'Wikipedia', 'query': 'https://en.wikipedia.org'}, {'intent': 'search', 'target': 'Wikipedia', 'query': 'Python programming'}]
Current state: {'user_query': 'Go to Wikipedia and search for Python programming', 'intent': [{'intent': 'navigate', 'target': 'Wikipedia', 'query': 'https://en.wikipedia.org'}, {'intent': 'search', 'target': 'Wikipedia', 'query': 'Python programming'}], 'action_plan': [], 'next_step': None, 'current_step': 0, 'current_url': None, 'dom_snapshot': {}, 'execution_trace': [], 'error': 